In [2]:
from pathlib import Path

# 바꿀 경로 정의
old_prefix = "/work/mech-ai-scratch/bgekim/project/imputation/IA_dataset/30m/Patches/"
new_prefix = "/scratch/bepk/bkim2/patches/30m/"

# 입력/출력 파일 경로
input_file = "/scratch/bepk/bkim2/MultiMAE/valid_list/nova/30m/pair_S1_paths.txt"
output_file = "/scratch/bepk/bkim2/MultiMAE/valid_list/delta/30m/pair_S1_paths.txt"

# 파일을 한 줄씩 읽어서 경로를 치환하고 새 파일에 씁니다.
with open(input_file, "r") as fin, open(output_file, "w") as fout:
    for line in fin:
        line = line.strip()
        if line:
            # 경로 치환
            new_line = line.replace(old_prefix, new_prefix)
            fout.write(new_line + "\n")


In [2]:
#!/usr/bin/env python3
"""
S1 vs MODIS·S2 경로 비교
- 세 txt 파일은 한 줄에 하나의 full‑path가 들어 있다고 가정
- 결과:
    1) 각 리스트 총 개수 출력
    2) MODIS·S2에는 없고 S1에만 있는 키 목록 출력
"""

from pathlib import Path

# -------------------------------------------------------------
MODIS_TXT = Path("valid_MODIS.txt")   # 경로 수정
S2_TXT    = Path("valid_S2.txt")
S1_TXT    = Path("valid_S1.txt")
# -------------------------------------------------------------

def read_lines(f: Path):
    return [ln.strip() for ln in f.read_text().splitlines() if ln.strip()]

def make_key(full_path: str) -> str:
    """
    '/.../patches/MODIS/patch_1008_1008/2018-07-22_IA.tif'
        → 'patch_1008_1008/2018-07-22_IA.tif'
    (모달리티 이름 제거: 패치폴더/파일명 만 남김)
    """
    parts = full_path.split("/patches/", 1)[-1].split("/", 2)
    # parts: [ 'MODIS', 'patch_1008_1008', '2018-07-22_IA.tif' ]
    return "/".join(parts[1:]) if len(parts) >= 3 else full_path

def load_keys(txt_path: Path):
    return {make_key(p) for p in read_lines(txt_path)}

# ---------- 실행 ----------
modis_keys = load_keys(MODIS_TXT)
s2_keys    = load_keys(S2_TXT)
s1_keys    = load_keys(S1_TXT)

print("----- 리스트 개수 -----")
print(f"MODIS: {len(modis_keys):>6}")
print(f"S2   : {len(s2_keys):>6}")
print(f"S1   : {len(s1_keys):>6}\n")

# 두 기준(MODIS·S2) 합쳐서 참조 집합
reference = modis_keys | s2_keys
extra_in_s1 = sorted(s1_keys - reference)     # S1에만 있는 경로

print(f"S1 전용(튀는) 항목: {len(extra_in_s1)}개")
for k in extra_in_s1[:20]:                    # 처음 20개 미리보기
    print("  ", k)

# 필요하면 파일로 저장
Path("extra_in_S1.txt").write_text("\n".join(extra_in_s1) + "\n")
print("\n✅  extra_in_S1.txt 파일로 저장 완료")


----- 리스트 개수 -----
MODIS:  60519
S2   :  61128
S1   :  60838

S1 전용(튀는) 항목: 949개
   patch_0_896/2016-05-01_IA.tif
   patch_0_896/2017-08-20_IA.tif
   patch_0_896/2017-09-10_IA.tif
   patch_0_896/2018-04-22_IA.tif
   patch_0_896/2018-05-20_IA.tif
   patch_0_896/2018-07-08_IA.tif
   patch_0_896/2018-07-22_IA.tif
   patch_0_896/2018-08-12_IA.tif
   patch_0_896/2018-09-16_IA.tif
   patch_0_896/2018-10-14_IA.tif
   patch_0_896/2018-10-21_IA.tif
   patch_0_896/2018-10-28_IA.tif
   patch_0_896/2019-04-14_IA.tif
   patch_0_896/2019-05-05_IA.tif
   patch_0_896/2019-05-12_IA.tif
   patch_0_896/2019-07-14_IA.tif
   patch_0_896/2019-07-21_IA.tif
   patch_0_896/2019-07-28_IA.tif
   patch_0_896/2019-08-04_IA.tif
   patch_0_896/2019-08-25_IA.tif

✅  extra_in_S1.txt 파일로 저장 완료
